# Generatívne siete

Rekurentné neurónové siete (RNN) a ich varianty s bránkovými bunkami, ako sú bunky s dlhodobou a krátkodobou pamäťou (LSTMs) a bránkové rekurentné jednotky (GRUs), poskytujú mechanizmus na modelovanie jazyka, t.j. dokážu sa naučiť poradie slov a poskytovať predpovede pre nasledujúce slovo v sekvencii. To nám umožňuje používať RNN pre **generatívne úlohy**, ako je bežná generácia textu, strojový preklad a dokonca aj popisovanie obrázkov.

V architektúre RNN, ktorú sme diskutovali v predchádzajúcej jednotke, každá jednotka RNN produkovala ako výstup nasledujúci skrytý stav. Avšak môžeme tiež pridať ďalší výstup ku každej rekurentnej jednotke, čo nám umožní generovať **sekvenciu** (ktorá má rovnakú dĺžku ako pôvodná sekvencia). Navyše môžeme použiť jednotky RNN, ktoré neprijímajú vstup na každom kroku, ale len berú nejaký počiatočný stavový vektor a potom produkujú sekvenciu výstupov.

V tomto notebooku sa zameriame na jednoduché generatívne modely, ktoré nám pomáhajú generovať text. Pre jednoduchosť vytvorme **sieť na úrovni znakov**, ktorá generuje text písmeno po písmene. Počas tréningu musíme vziať nejaký textový korpus a rozdeliť ho na sekvencie písmen.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Vytváranie slovníka znakov

Na vytvorenie generatívnej siete na úrovni znakov je potrebné rozdeliť text na jednotlivé znaky namiesto slov. Vrstva `TextVectorization`, ktorú sme používali predtým, to nedokáže, takže máme dve možnosti:

* Ručne načítať text a vykonať tokenizáciu „ručne“, ako je uvedené v [tomto oficiálnom príklade Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Použiť triedu `Tokenizer` na tokenizáciu na úrovni znakov.

Zvolíme druhú možnosť. `Tokenizer` sa dá použiť aj na tokenizáciu do slov, takže by malo byť pomerne jednoduché prepnúť z tokenizácie na úrovni znakov na tokenizáciu na úrovni slov.

Na vykonanie tokenizácie na úrovni znakov je potrebné odovzdať parameter `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Chceme tiež použiť jeden špeciálny token na označenie **konca sekvencie**, ktorý nazveme `<eos>`. Pridajme ho manuálne do slovníka:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Trénovanie generatívnej RNN na generovanie nadpisov

Spôsob, akým budeme trénovať RNN na generovanie nadpisov správ, je nasledovný. V každom kroku vezmeme jeden nadpis, ktorý bude poskytnutý RNN, a pre každý vstupný znak požiadame sieť, aby vygenerovala nasledujúci výstupný znak:

![Obrázok zobrazujúci príklad generovania slova 'HELLO' pomocou RNN.](../../../../../translated_images/sk/rnn-generate.56c54afb52f9781d.webp)

Pre posledný znak našej sekvencie požiadame sieť, aby vygenerovala token `<eos>`.

Hlavný rozdiel medzi generatívnou RNN, ktorú tu používame, je ten, že vezmeme výstup z každého kroku RNN, a nie len z poslednej bunky. To je možné dosiahnuť nastavením parametra `return_sequences` pre bunku RNN.

Takže počas trénovania bude vstupom do siete sekvencia zakódovaných znakov určitej dĺžky a výstupom bude sekvencia rovnakej dĺžky, ale posunutá o jeden prvok a ukončená tokenom `<eos>`. Minibatch bude pozostávať z viacerých takýchto sekvencií, a bude potrebné použiť **padding**, aby sa všetky sekvencie zarovnali.

Vytvorme funkcie, ktoré nám transformujú dataset. Pretože chceme sekvencie dopĺňať na úrovni minibatchu, najskôr dataset zoskupíme pomocou `.batch()`, a potom ho upravíme pomocou `map`, aby sme vykonali transformáciu. Transformačná funkcia teda bude brať celý minibatch ako parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Niekoľko dôležitých vecí, ktoré tu robíme:
* Najskôr extrahujeme skutočný text zo stringového tenzora
* `text_to_sequences` konvertuje zoznam reťazcov na zoznam celočíselných tenzorov
* `pad_sequences` doplní tieto tenzory na ich maximálnu dĺžku
* Nakoniec všetky znaky zakódujeme pomocou one-hot kódovania, a tiež vykonáme posun a pridanie `<eos>`. Čoskoro uvidíme, prečo potrebujeme znaky zakódované pomocou one-hot.

Táto funkcia je však **Pythonická**, t.j. nemôže byť automaticky preložená do výpočtového grafu Tensorflow. Ak sa pokúsime použiť túto funkciu priamo vo funkcii `Dataset.map`, dostaneme chyby. Musíme túto Pythonickú výzvu obaliť použitím obalu `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Poznámka**: Rozlíšiť medzi Pythonickými a Tensorflow transformačnými funkciami sa môže zdať príliš zložité, a možno sa pýtate, prečo dataset netransformujeme pomocou štandardných Python funkcií predtým, ako ho odovzdáme do `fit`. Aj keď to určite možné je, použitie `Dataset.map` má obrovskú výhodu, pretože dátová transformačná pipeline je vykonávaná pomocou Tensorflow výpočtového grafu, ktorý využíva výpočty na GPU a minimalizuje potrebu prenášať dáta medzi CPU a GPU.

Teraz môžeme vytvoriť našu generátorovú sieť a začať trénovať. Môže byť založená na akejkoľvek rekurentnej bunke, ktorú sme preberali v predchádzajúcej jednotke (jednoduchá, LSTM alebo GRU). V našom príklade použijeme LSTM.

Keďže sieť prijíma znaky ako vstup a veľkosť slovníka je pomerne malá, nepotrebujeme embedding vrstvu, jednohotovo zakódovaný vstup môže ísť priamo do LSTM bunky. Výstupná vrstva bude `Dense` klasifikátor, ktorý premení výstup LSTM na jednohotovo zakódované čísla tokenov.

Okrem toho, keďže pracujeme so sekvenciami s premenlivou dĺžkou, môžeme použiť vrstvu `Masking` na vytvorenie masky, ktorá bude ignorovať doplnené časti reťazca. Toto nie je striktne potrebné, pretože nás veľmi nezaujíma všetko, čo presahuje token `<eos>`, ale použijeme to kvôli získaniu skúseností s týmto typom vrstvy. `input_shape` bude `(None, vocab_size)`, kde `None` označuje sekvenciu s premenlivou dĺžkou, a výstupný tvar je taktiež `(None, vocab_size)`, ako môžete vidieť zo `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generovanie výstupu

Teraz, keď sme model natrénovali, chceme ho použiť na generovanie výstupu. V prvom rade potrebujeme spôsob, ako dekódovať text reprezentovaný sekvenciou čísel tokenov. Na tento účel by sme mohli použiť funkciu `tokenizer.sequences_to_texts`; tá však nefunguje dobre s tokenizáciou na úrovni znakov. Preto si vezmeme slovník tokenov z tokenizéra (nazývaný `word_index`), vytvoríme reverznú mapu a napíšeme si vlastnú dekódovaciu funkciu:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Teraz začneme generovanie. Začneme s nejakým reťazcom `start`, zakódujeme ho do sekvencie `inp`, a potom v každom kroku zavoláme našu sieť, aby sme odvodili nasledujúci znak.

Výstup siete `out` je vektor s `vocab_size` prvkami, ktoré predstavujú pravdepodobnosti každého tokenu, a najpravdepodobnejší token môžeme nájsť pomocou `argmax`. Tento znak potom pridáme do generovaného zoznamu tokenov a pokračujeme v generovaní. Tento proces generovania jedného znaku sa opakuje `size` krát, aby sme vygenerovali požadovaný počet znakov, a predčasne ukončíme, keď sa objaví `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Vzorkovanie výstupu počas tréningu

Keďže nemáme žiadne užitočné metriky, ako napríklad *presnosť*, jediný spôsob, ako môžeme vidieť, že náš model sa zlepšuje, je **vzorkovanie** generovaného reťazca počas tréningu. Na tento účel použijeme **callbacky**, teda funkcie, ktoré môžeme odovzdať funkcii `fit` a ktoré budú pravidelne volané počas tréningu.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Tento príklad už generuje pomerne dobrý text, ale dá sa ešte vylepšiť viacerými spôsobmi:
* **Viac textu**. Použili sme iba nadpisy pre našu úlohu, ale môžete experimentovať s plným textom. Pamätajte, že RNN nie sú príliš dobré pri spracovaní dlhých sekvencií, takže má zmysel buď rozdeliť text na kratšie vety, alebo vždy trénovať na pevnej dĺžke sekvencie s nejakou preddefinovanou hodnotou `num_chars` (napríklad 256). Môžete skúsiť zmeniť vyššie uvedený príklad na takúto architektúru, pričom sa môžete inšpirovať [oficiálnym tutoriálom Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/).
* **Viacvrstvové LSTM**. Má zmysel vyskúšať 2 alebo 3 vrstvy LSTM buniek. Ako sme spomenuli v predchádzajúcej jednotke, každá vrstva LSTM extrahuje určité vzory z textu, a v prípade generátora na úrovni znakov môžeme očakávať, že nižšia úroveň LSTM bude zodpovedná za extrahovanie slabík, a vyššie úrovne - za slová a kombinácie slov. Toto sa dá jednoducho implementovať pomocou parametra počtu vrstiev v konštruktore LSTM.
* Môžete tiež experimentovať s **GRU jednotkami** a zistiť, ktoré fungujú lepšie, a s **rôznymi veľkosťami skrytých vrstiev**. Príliš veľká skrytá vrstva môže viesť k pretrénovaniu (napr. sieť sa naučí presný text), a menšia veľkosť nemusí produkovať dobrý výsledok.


## Generovanie mäkkého textu a teplota

V predchádzajúcej definícii `generate` sme vždy vyberali znak s najvyššou pravdepodobnosťou ako nasledujúci znak v generovanom texte. To malo za následok, že text často "cykloval" medzi rovnakými sekvenciami znakov znova a znova, ako v tomto príklade:
```
today of the second the company and a second the company ...
```

Ak sa však pozrieme na rozdelenie pravdepodobností pre nasledujúci znak, môže sa stať, že rozdiel medzi niekoľkými najvyššími pravdepodobnosťami nie je veľký, napríklad jeden znak môže mať pravdepodobnosť 0,2, iný 0,19 atď. Napríklad pri hľadaní nasledujúceho znaku v sekvencii '*play*' môže byť nasledujúcim znakom rovnako dobre medzera alebo **e** (ako v slove *player*).

To nás vedie k záveru, že nie je vždy "spravodlivé" vybrať znak s vyššou pravdepodobnosťou, pretože výber druhého najvyššieho môže stále viesť k zmysluplnému textu. Je rozumnejšie **vzorkovať** znaky z rozdelenia pravdepodobností, ktoré poskytuje výstup siete.

Toto vzorkovanie je možné vykonať pomocou funkcie `np.multinomial`, ktorá implementuje takzvané **multinomiálne rozdelenie**. Funkcia, ktorá implementuje toto **mäkké** generovanie textu, je definovaná nižšie:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Zaviedli sme ďalší parameter nazývaný **teplota**, ktorý sa používa na určenie, ako striktne by sme sa mali držať najvyššej pravdepodobnosti. Ak je teplota 1.0, vykonávame spravodlivé multinomiálne vzorkovanie, a keď teplota dosiahne nekonečno - všetky pravdepodobnosti sa stanú rovnakými a náhodne vyberáme ďalší znak. V príklade nižšie môžeme pozorovať, že text sa stáva nezmyselným, keď príliš zvýšime teplotu, a pripomína „cyklický“ ťažko generovaný text, keď sa teplota blíži k 0.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
